In [1]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd

In [2]:
O=pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Ordering',usecols="C:IH").values
S=pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="C:IH").values
MT=np.diag(1./(pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="B").values.flatten()))
O=np.dot(MT,O)
S=np.dot(MT,S)

In [3]:
feature_1 = ((S != 0).sum(axis = 1)/240)
feature_2 = np.cbrt(np.max(S,axis=1)/np.max(S))
a = (O-S)/O
a[np.isinf(a)] = 0
a[np.isnan(a)] = 0
feature_3 = np.abs(1-np.average(np.abs(a),axis=1))
feature_4 = np.sqrt(np.average(S,axis=1)/np.max(S,axis=1))

In [4]:
Z=np.vstack((feature_1,feature_2,feature_3,feature_4)).T
cmin=np.min(Z,axis=0)
cmax=np.max(Z,axis=0)
cmaxmin=cmax-cmin
Z=(Z-cmin)/cmaxmin
P=Z/np.sum(Z,axis=0)
log_P=np.log(P)
log_P[np.isinf(log_P)] = 0
log_P[np.isnan(log_P)] = 0
e=np.zeros(4)
for i in [0,1,2,3]:
    e[i]=np.dot(P[...,i].T,log_P[...,i])
g=1-(-e/np.log(402))
w=g/np.sum(g)

In [5]:
r=np.corrcoef(Z.T)
sigma=np.std(Z,axis=0)
d=4*np.ones(4)-np.sum(r,axis=0)
d=(g+sigma)*d
w=d/np.sum(d)

In [6]:
b=Z/np.linalg.norm(Z,axis=0)
dream=np.max(b,axis=0)
nodr=np.min(b,axis=0)
sdream=np.linalg.norm(np.sqrt(w)*(b-dream),axis=1)
snodr=np.linalg.norm(np.sqrt(w)*(b-nodr),axis=1)
score=snodr/(sdream+snodr)
df=np.sort(score)[::-1]
ind=score.argsort()[::-1]